<a href="https://colab.research.google.com/github/steysie/Computational-Linguistics-Homeworks/blob/master/RussianNumberConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Russian Number Converter
#### Anastasia Nikiforova

In [0]:
!pip install num2words

In [0]:
!pip install pymorphy2

In [0]:
import re
import pymorphy2
from spacy.lang.ru import Russian
from num2words import num2words
morph = pymorphy2.MorphAnalyzer()

In [0]:
class RussianNumberConverter:
    '''
    The Russian Number Converter inputs and outputs a string.
    It is fully rule-based, and uses annotated Open Corpora via pymorphy2.
    Rules can be modified and supplemented if needed. Existing rules cover most rules of number agreement in Rusian language.
    It is suggested to feed sentence-length segments to the converter.

    Example input:  "нет 1928 билетов"
    Example output: "нет одной тысячи девятисот двадцати восьми билетов"
    '''

    def __init__(self):
        self.new_string = ""

    def tokenize(self, text):
        ''' 
        Spacy tokenizer.
        Each word and puctuation mark is a separate token.
        '''
        nlp = Russian()
        doc = nlp(text)

        return [token.text for token in doc]

    def convert(self, text):

        tmp = self.tokenize(text)

        for tok_n in range(len(tmp)):
            if re.match(r"\d+", tmp[tok_n]):
                raw_num = num2words(tmp[tok_n], lang="ru")
                
                # результат конвертации может быть составным числительным 
                raw_num_split = raw_num.split()
                raw_num_tagged = [morph.parse(i)[0] for i in raw_num_split]

                if tok_n>0 and tmp[tok_n-1] == "нет":

                    # поставить числительное в форму Родительного падежа
                    raw_num_split = [token.inflect({"gent"}).word for token in raw_num_tagged]

                elif tok_n>0 and tmp[tok_n-1] == "c":

                    # поставить числительное в форму Предложного падежа
                    raw_num_split = [token.inflect({"ablt"}).word for token in raw_num_tagged]

                # next/one-of-the-next word dependency
                # check if there are any nouns or adjectives in the rest of the sentence

                elif tok_n != len(tmp)-1 and any(tag in [morph.parse(i)[0].tag.POS for i in tmp[tok_n+1:]] for tag in ["NOUN", "ADJF"]):
                    # согласовать с их родом и числом (inflect)

                    next_tok = next(morph.parse(tok)[0] for tok in tmp[tok_n+1:] if morph.parse(tok)[0].tag.POS in ["NOUN", "ADJF"])
                    # print(next_tok.word)
                    
                    if raw_num_split[-1] in ["один", "два"]:
                        # для чисел, оканцивающихся на "один" и "два" меняем только последнее слово

                        # если зависимое слово - существительное, согласуем по роду и падежу
                        if morph.parse(tmp[tok_n+1])[0].tag.POS == "NOUN":
                            # смотрим на одушевленность
                            if next_tok.tag.animacy == "inan":
                                if next_tok.tag.gender:
                                    raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.gender, next_tok.tag.case}).word

                                if raw_num_tagged[-1].tag.animacy:
                                    raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.animacy, next_tok.tag.case}).word
                                
                                elif next_tok.tag.case == "gent":
                                    if next_tok.tag.gender:
                                        raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.gender, "accs"}).word
                                    else:
                                        raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.animacy, "accs"}).word

                                else:
                                    raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.case}).word
                            
                            else:
                                raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.gender, next_tok.tag.case}).word
                        
                        # если зависимое слово - прилагательное, согласуем по падежу
                        if next_tok.tag.POS == "ADJF":
                            if next_tok.tag.gender:
                                raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.case, next_tok.tag.case}).word

                            else:
                                raw_num_split[-1] = raw_num_tagged[-1].inflect({next_tok.tag.case}).word

                    #### noun == "gent" --> numr == "accs"
                    elif next_tok.tag.case == "gent":

                        raw_num_split = [token.inflect({'accs'}).word for token in raw_num_tagged]

                    else:     
                        raw_num_split = [token.inflect({next_tok.tag.case}).word for token in raw_num_tagged]

                elif tok_n>0 and morph.parse(tmp[tok_n-1])[0].tag.POS == "PREP":

                    # поставить числительное в форму Предложного падежа
                    raw_num_split = [token.inflect({"loct"}).word for token in raw_num_tagged]

                # если предыдущие условия не подходят, оставляем как есть - предполагаем, что таких случаев незначительно мало и именительный падеж подходит
                accord_number = " ".join([i for i in raw_num_split])
                tmp[tok_n] = accord_number
        
        self.new_string = " ".join(text for text in tmp)

        return self.new_string


In [0]:
conv = RussianNumberConverter()

In [9]:
s = ["5 котов",
     "1 котом",
     "10 котами",
     "2 больших капибар",
     "о 55 гномах",
     "41 чертенку",
     "1 маленькой черепашки",
     "2 стола", 
     "18 мест",
     "нет 18 мест",
     "нет 1928 билетов",
     "8 столами",
     "9 котами", 
     "c 274 карандашами",
     "о 2",
     "c 2"]

[conv.convert(i) for i in s]

['пять котов',
 'одним котом',
 'десятью котами',
 'двух больших капибар',
 'о пятидесяти пяти гномах',
 'сорок одному чертенку',
 'одной маленькой черепашки',
 'два стола',
 'восемнадцать мест',
 'нет восемнадцати мест',
 'нет одной тысячи девятисот двадцати восьми билетов',
 'восемью столами',
 'девятью котами',
 'c двумястами семьюдесятью четырьмя карандашами',
 'о двух',
 'c двумя']